In [69]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from glob import glob
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [158]:
user_list = glob('./data/source/users/*')[:20]
promotion_list = glob('./data/source/promotions/*')[:20]
transactions_list = glob('./data/source/transactions/*')[:20]

In [9]:
spark = (SparkSession
        .builder
        .appName("mock project")
        .getOrCreate())

In [52]:
for file in user_list:
    user = spark.read.csv(file, sep='\t', header=True)
    user.write.parquet('./Spark_mock_project/datalake/user/' + file.split('\\')[-1])

for file in promotion_list:
    prom = spark.read.csv(file, sep='\t', header=True)
    prom.write.parquet('./Spark_mock_project/datalake/promotion/' + file.split('\\')[-1])

for file in transactions_list:
    tran = spark.read.csv(file, sep='\t', header=True)
    tran.write.parquet('./Spark_mock_project/datalake/transaction/' + file.split('\\')[-1])

## User demography

### 1. Integrate

In [96]:
user = (spark.read
        .option("header","true")
        .option("recursiveFileLookup","true")
        .parquet("./Spark_mock_project/datalake/user")
        .withColumn("updatedTime", to_date(col("updatedTime"), "yyyy-MM-dd HH:mm:ss.SSS"))
        .withColumn("birthdate", to_date(col("birthdate")))
        .withColumn("Age", floor(datediff(current_date(), col("birthdate"))/365.25)))

In [97]:
user.show(3)

+------+----------+------------+------+-----------+---+
|userid| birthdate|profileLevel|gender|updatedTime|Age|
+------+----------+------------+------+-----------+---+
| 11176|1962-06-28|           3|     2| 2021-11-02| 59|
| 31287|1964-06-28|           1|     2| 2021-11-02| 57|
| 32183|1982-02-17|           3|     1| 2021-11-02| 40|
+------+----------+------------+------+-----------+---+
only showing top 3 rows



In [86]:
w = Window.partitionBy(user.userid).orderBy(user.updatedTime.desc())
user_df = user.withColumn("row_number", row_number().over(w)).filter(col("row_number")==1).drop("row_number")

In [88]:
user.count(), user_df.count()

(8940, 8114)

### 2. Update record

In [159]:
new_user = (spark.read
        .option("header","true")
        # .option("recursiveFileLookup","true")
        .parquet("./Spark_mock_project/datalake/test/user/2021-11-21")
        .withColumn("updatedTime", to_date(col("updatedTime"), "yyyy-MM-dd HH:mm:ss.SSS"))
        .withColumn("birthdate", to_date(col("birthdate")))
        .withColumn("Age", floor(datediff(current_date(), col("birthdate"))/365.25)))

In [164]:
final_user = user_df.union(new_user)
final_user.count()

8566

In [165]:
w = Window.partitionBy(final_user.userid).orderBy(final_user.updatedTime.desc())
final_user = final_user.withColumn("row_number", row_number().over(w)).filter(col("row_number")==1).drop("row_number")

In [183]:
final_user = final_user.withColumn("Age", floor(datediff(current_date(), col("birthdate"))/365.25))

In [184]:
final_user.count()

8482

## User payment

### 1. Integrate

In [91]:
tran = (spark.read
        .option("header","true")
        .option("recursiveFileLookup","true")
        .parquet("./Spark_mock_project/datalake/transaction")
        .filter("transStatus = 1")
        .withColumn("transactionTime", unix_timestamp(col("transactionTime"), "yyyy-MM-dd HH:mm:ss.SSS").cast("timestamp")))

In [93]:
tran.show(3)

+--------------------+-----------+------+-------------------+-----+---------+-------+-----+
|             transId|transStatus|userId|    transactionTime|appId|transType| amount|pmcId|
+--------------------+-----------+------+-------------------+-----+---------+-------+-----+
|20211105-81405105...|          1|  8562|2021-11-05 16:53:28|    5|        5|8709973|    2|
|20211105-56578372...|          1| 22580|2021-11-05 06:47:11|    1|        1|1860312|    3|
|20211105--6905617...|          1| 22781|2021-11-05 10:14:18|    5|        5|6527582|    2|
+--------------------+-----------+------+-------------------+-----+---------+-------+-----+
only showing top 3 rows



In [98]:
paymentDate = (tran
    .filter("transType = 3")
    .groupBy("userId")
    .agg(min("transactionTime").alias("fistPaymentDate"), max("transactionTime").alias("transactionTime")))

In [99]:
payment = (paymentDate.join(tran.filter("transType = 3"), ["userId", "transactionTime"])
    .withColumnRenamed("transactionTime", "lastPaymentDate")
    .withColumnRenamed("appId", "lastPaymentApp")
    .select("userId", "fistPaymentDate", "lastPaymentDate", "lastPaymentApp"))

In [100]:
payment.show(3)

+------+-------------------+-------------------+--------------+
|userId|    fistPaymentDate|    lastPaymentDate|lastPaymentApp|
+------+-------------------+-------------------+--------------+
| 11992|2021-11-05 09:51:52|2021-11-05 09:51:52|           107|
| 28874|2021-11-05 05:09:38|2021-11-05 22:01:10|            92|
| 30413|2021-11-05 06:09:54|2021-11-05 06:09:54|            58|
+------+-------------------+-------------------+--------------+
only showing top 3 rows



### 2. Update record

In [178]:
new_tran = (spark.read
        .option("header","true")
        # .option("recursiveFileLookup","true")
        .parquet("./Spark_mock_project/datalake/test/transaction/2021-11-21")
        .filter("transStatus = 1")
        .withColumn("transactionTime", unix_timestamp(col("transactionTime"), "yyyy-MM-dd HH:mm:ss.SSS").cast("timestamp")))

In [179]:
new_payment = (new_tran
    .filter("transType = 3")
    .withColumn("fistPaymentDate", col("transactionTime"))
    .withColumnRenamed("transactionTime", "lastPaymentDate")
    .withColumnRenamed("appId", "lastPaymentApp")
    .select("userId", "fistPaymentDate", "lastPaymentDate", "lastPaymentApp"))

In [176]:
new_payment.show(3)

+------+--------------------+--------------------+--------------+
|userId|     fistPaymentDate|     lastPaymentDate|lastPaymentApp|
+------+--------------------+--------------------+--------------+
| 16893|2021-11-22 01:59:...|2021-11-22 01:59:...|           106|
|  2604|2021-11-22 11:23:...|2021-11-22 11:23:...|            65|
| 12280|2021-11-22 09:44:...|2021-11-22 09:44:...|            19|
+------+--------------------+--------------------+--------------+
only showing top 3 rows



In [189]:
union_payment = payment.union(new_payment)

In [204]:
w = Window.partitionBy("userId")
final_payment = (union_payment.withColumn("firstDate", min("fistPaymentDate").over(w))
    .withColumn("lastDate", max("lastPaymentDate").over(w))
    .filter("lastPaymentDate = lastDate")
    .select("userId", "firstDate", "lastDate", "lastPaymentApp")
    .withColumnRenamed("firstDate", "fistPaymentDate")
    .withColumnRenamed("lastDate", "lastPaymentDate"))

In [205]:
final_payment.show(3)

+------+-------------------+-------------------+--------------+
|userId|    fistPaymentDate|    lastPaymentDate|lastPaymentApp|
+------+-------------------+-------------------+--------------+
| 18130|2021-11-07 12:37:18|2021-11-07 12:37:18|            60|
| 18574|2021-11-10 03:53:22|2021-11-10 03:53:22|            93|
| 18992|2021-11-15 00:12:41|2021-11-15 00:12:41|            23|
+------+-------------------+-------------------+--------------+
only showing top 3 rows



In [207]:
payment.count(), new_payment.count(), union_payment.count(), final_payment.count()

(4121, 238, 4359, 4335)

## User activity

### 1. Integrate

In [102]:
activeDate = (tran
    .groupBy("userId")
    .agg(min("transactionTime").alias("fistActiveDate"), max("transactionTime").alias("transactionTime")))

In [212]:
activeDate.show(3)

+------+-------------------+-------------------+
|userId|     fistActiveDate|    transactionTime|
+------+-------------------+-------------------+
| 19338|2021-11-05 07:45:22|2021-11-05 07:45:22|
| 16250|2021-11-05 12:45:59|2021-11-05 12:45:59|
|  3210|2021-11-03 04:29:52|2021-11-05 08:09:41|
+------+-------------------+-------------------+
only showing top 3 rows



In [106]:
active = (activeDate.join(tran, ["userId", "transactionTime"])
    .withColumnRenamed("transactionTime", "lastActiveDAte")
    .withColumnRenamed("appId", "lastApp")
    .withColumnRenamed("transType", "lastTransType")
    .select("userId", "fistActiveDate", "lastActiveDAte", "lastTransType"))

In [218]:
active.show(3)

+------+-------------------+-------------------+-------------+
|userId|     fistActiveDate|     lastActiveDAte|lastTransType|
+------+-------------------+-------------------+-------------+
|  8562|2021-11-05 16:53:28|2021-11-05 16:53:28|            5|
|  3999|2021-11-05 19:51:37|2021-11-05 19:51:37|            2|
| 11992|2021-11-05 09:51:52|2021-11-05 09:51:52|            3|
+------+-------------------+-------------------+-------------+
only showing top 3 rows



### 2. Update record

In [216]:
new_active = (new_tran
    .withColumn("fistActiveDate", col("transactionTime"))
    .withColumnRenamed("transactionTime", "lastActiveDAte")
    .withColumnRenamed("transType", "lastTransType")
    .select("userId", "fistActiveDate", "lastActiveDAte", "lastTransType"))

In [215]:
union_active = active.union(new_active)

In [220]:
w = Window.partitionBy("userId")
final_active = (union_active.withColumn("firstDate", min("fistActiveDate").over(w))
    .withColumn("lastDate", max("lastActiveDAte").over(w))
    .filter("lastActiveDAte = lastDate")
    .select("userId", "firstDate", "lastDate", "lastTransType")
    .withColumnRenamed("firstDate", "fistActiveDate")
    .withColumnRenamed("lastDate", "lastActiveDAte"))

In [221]:
final_active.show(3)

+------+-------------------+-------------------+-------------+
|userId|     fistActiveDate|     lastActiveDAte|lastTransType|
+------+-------------------+-------------------+-------------+
| 10096|2021-11-09 10:11:05|2021-11-09 10:11:05|            2|
|  1159|2021-11-21 00:07:03|2021-11-21 00:07:03|            5|
| 13282|2021-11-06 03:22:36|2021-11-06 03:22:36|            4|
+------+-------------------+-------------------+-------------+
only showing top 3 rows



In [222]:
active.count(), new_active.count(), union_active.count(), final_active.count()

(17080, 1105, 18185, 17729)

## App list

### 1. Integrate

In [108]:
userApp = tran.select("userId", "appId").distinct()

In [111]:
userApp.show(3)

+------+-----+
|userId|appId|
+------+-----+
| 38849|    5|
|  2945|    5|
| 23198|    4|
+------+-----+
only showing top 3 rows



### 2. Update record

In [229]:
new_userApp = new_tran.select("userId", "appId")
union_userApp = userApp.union(new_userApp)
final_userApp = union_userApp.distinct()

In [226]:
userApp.count(), new_userApp.count(), union_userApp.count(), final_userApp.count()

(21208, 1105, 22313, 22226)

## Source payment

### 1. Integrate

In [112]:
userPmc = tran.select("userId", "pmcId").distinct()

In [113]:
userPmc.show(3)

+------+-----+
|userId|pmcId|
+------+-----+
| 39053|    4|
| 20012|    1|
| 29482|    1|
+------+-----+
only showing top 3 rows



### 2. Update record

In [230]:
new_userPmc = new_tran.select("userId", "pmcId")
union_userPmc = userPmc.union(new_userPmc)
final_userPmc = union_userPmc.distinct()

In [231]:
userPmc.count(), new_userPmc.count(), union_userApp.count(), final_userPmc.count()

(20663, 1105, 22313, 21609)

## Promotion

### 1. Integrate

In [145]:
prom = (spark.read
        .option("header","true")
        .option("recursiveFileLookup","true")
        .parquet("./Spark_mock_project/datalake/promotion")
        .withColumn("time", unix_timestamp(col("time"), "yyyy-MM-dd HH:mm:ss.SSS")))

In [146]:
prom.show(3)

+------+-----------+------+----------+----------+
|userid|voucherCode|status|campaignID|      time|
+------+-----------+------+----------+----------+
| 18710|    1005-43| GIVEN|      1005|1636932816|
| 12966|    1001-60| GIVEN|      1001|1636924987|
|  7302|   1000-119| GIVEN|      1000|1636992179|
+------+-----------+------+----------+----------+
only showing top 3 rows



In [138]:
camp = (spark.read
    .option("sep", "\t")
    .option("header", "true")
    .csv("./Spark_mock_project/datalake/campaign.csv")
    .withColumn("expireDate", unix_timestamp(col("expireDate"), "yyyy-MM-dd HH:mm:ss").cast("timestamp"))
    .withColumn("expireTime", col("expireTime").cast("Integer")))

In [140]:
camp.show(3)

+----------+------------+-------------------+----------+
|campaignID|campaignType|         expireDate|expireTime|
+----------+------------+-------------------+----------+
|      1000|           1|2022-01-01 00:00:00|         0|
|      1001|           1|2022-01-01 00:00:00|         0|
|      1002|           1|2022-01-01 00:00:00|         0|
+----------+------------+-------------------+----------+
only showing top 3 rows



In [147]:
promotion = (prom.join(camp, "campaignID")
    .withColumn("voucherExpire", (col("time") + col("expireTime")).cast("timestamp"))
    .withColumn("time", col("time").cast("timestamp")))

In [148]:
promotion.show(3)

+----------+------+-----------+------+-------------------+------------+-------------------+----------+-------------------+
|campaignID|userid|voucherCode|status|               time|campaignType|         expireDate|expireTime|      voucherExpire|
+----------+------+-----------+------+-------------------+------------+-------------------+----------+-------------------+
|      1005| 18710|    1005-43| GIVEN|2021-11-15 06:33:36|           1|2022-01-01 00:00:00|         0|2021-11-15 06:33:36|
|      1001| 12966|    1001-60| GIVEN|2021-11-15 04:23:07|           1|2022-01-01 00:00:00|         0|2021-11-15 04:23:07|
|      1000|  7302|   1000-119| GIVEN|2021-11-15 23:02:59|           1|2022-01-01 00:00:00|         0|2021-11-15 23:02:59|
+----------+------+-----------+------+-------------------+------------+-------------------+----------+-------------------+
only showing top 3 rows



In [233]:
userPromotion = (promotion.withColumn("actualExpire", when((col("campaignType")=="2") & 
                                                              (col("expireDate") > col("voucherExpire")), col("voucherExpire"))
                     .otherwise(col("expireDate")))
    .withColumn("Expire", current_date() > col("actualExpire"))
    .select("userid", "campaignID", "status", "Expire"))

In [244]:
userPromotion.count()

19947

In [245]:
userPromotion = userPromotion.distinct()

In [246]:
userPromotion.count()

18998

In [247]:
userPromotion.show(3)

+------+----------+------+------+
|userid|campaignID|status|Expire|
+------+----------+------+------+
|  8057|      1006| GIVEN|  true|
| 14268|      1000| GIVEN|  true|
| 16801|      1003| GIVEN|  true|
+------+----------+------+------+
only showing top 3 rows



### 2. Update record

In [235]:
new_prom = (spark.read
        .option("header","true")
        .parquet("./Spark_mock_project/datalake/test/promotion/2021-11-21")
        .withColumn("time", unix_timestamp(col("time"), "yyyy-MM-dd HH:mm:ss.SSS")))

In [236]:
new_prom.show(3)

+------+-----------+------+----------+----------+
|userid|voucherCode|status|campaignID|      time|
+------+-----------+------+----------+----------+
|  6279|     1003-1| GIVEN|      1003|1637489477|
| 15400|     1006-2| GIVEN|      1006|1637477346|
|  9156|    1004-35| GIVEN|      1004|1637513984|
+------+-----------+------+----------+----------+
only showing top 3 rows



In [138]:
camp = (spark.read
    .option("sep", "\t")
    .option("header", "true")
    .csv("./Spark_mock_project/datalake/campaign.csv")
    .withColumn("expireDate", unix_timestamp(col("expireDate"), "yyyy-MM-dd HH:mm:ss").cast("timestamp"))
    .withColumn("expireTime", col("expireTime").cast("Integer")))

In [237]:
new_promotion = (new_prom.join(camp, "campaignID")
    .withColumn("voucherExpire", (col("time") + col("expireTime")).cast("timestamp"))
    .withColumn("time", col("time").cast("timestamp")))

In [238]:
new_promotion.show(3)

+----------+------+-----------+------+-------------------+------------+-------------------+----------+-------------------+
|campaignID|userid|voucherCode|status|               time|campaignType|         expireDate|expireTime|      voucherExpire|
+----------+------+-----------+------+-------------------+------------+-------------------+----------+-------------------+
|      1003|  6279|     1003-1| GIVEN|2021-11-21 17:11:17|           1|2022-01-01 00:00:00|         0|2021-11-21 17:11:17|
|      1006| 15400|     1006-2| GIVEN|2021-11-21 13:49:06|           2|2022-01-01 00:00:00|   6048000|2022-01-30 13:49:06|
|      1004|  9156|    1004-35| GIVEN|2021-11-21 23:59:44|           1|2022-01-01 00:00:00|         0|2021-11-21 23:59:44|
+----------+------+-----------+------+-------------------+------------+-------------------+----------+-------------------+
only showing top 3 rows



In [248]:
new_userPromotion = (new_promotion.withColumn("actualExpire", when((col("campaignType")=="2") & 
                                                              (col("expireDate") > col("voucherExpire")), col("voucherExpire"))
                     .otherwise(col("expireDate")))
    .withColumn("Expire", current_date() > col("actualExpire"))
    .select("userid", "campaignID", "status", "Expire"))

In [249]:
union_userPromotion = userPromotion.union(new_userPromotion)
final_userPromotion = union_userPromotion.distinct()

In [250]:
final_userPromotion.show(3)

+------+----------+------+------+
|userid|campaignID|status|Expire|
+------+----------+------+------+
|  8057|      1006| GIVEN|  true|
| 14268|      1000| GIVEN|  true|
| 16801|      1003| GIVEN|  true|
+------+----------+------+------+
only showing top 3 rows



In [251]:
userPromotion.count(), new_userPromotion.count(), union_userPromotion.count(), final_userPromotion.count()

(18998, 971, 19969, 19879)